In [22]:
def remove_dup(lst: list) -> list:
    return list(set(lst))

remove_dup([5,4,1,5,4,2,1,3])

[1, 2, 3, 4, 5]

In [23]:
def remove_dup_1(lst: list) -> list:
    return list(dict.fromkeys(lst))


remove_dup_1([5,4,1,5,4,2,1,3])

[5, 4, 1, 2, 3]

In [24]:
def remove_dup_2(lst: list) -> list:
    seen = set()
    result = []
    for i in lst:
        if i not in seen:
            seen.add(i)
            result.append(i)

    return result


remove_dup_2([5,4,1,5,4,2,1,3])

[5, 4, 1, 2, 3]

In [27]:
from typing import Optional, Set, Iterable, List

class DataStream:
    def __init__(self) -> None:
        self.n: int = 0
        self.total: float = 0
        self.curr_max: Optional[int | float] = None
        self.freq: dict[int | float] = {}
        self.max_freq: int = 0
        self.modes: Set[int | float] = set()

    def insert(self, x: int | float) -> None:
        self.n += 1
        self.total += x

        if self.curr_max is None or x > self.curr_max:
            self.curr_max = x

        c = self.freq.get(x,0) + 1
        self.freq[x] = c 

        if c > self.max_freq:
            self.max_freq = c 
            self.modes = {x}
        elif c == self.max_freq:
            self.modes.add(x)

    def insert_many(self, xs: Iterable[int | float]) -> None:
        for x in xs:
            self.insert(x)

    def mean(self) -> Optional[float]:
        return None if self.n == 0 else self.total/self.n
    
    def max(self) -> Optional[int | float]:
        return self.curr_max
    
    def modes_all(self) -> List[int | float]:
        return list(self.modes)
    
s = DataStream()
s.insert_many([3,1,2,3,2,2,5])
print(s.mean())
print(s.max())
print(s.modes_all())


2.5714285714285716
5
[2]


In [40]:
from typing import Optional, Set, Iterable, List, Deque
from collections import deque, defaultdict

class DataStream:
    def __init__(self, k) -> None:
        self.n: int = 0
        self.total: float = 0
        self.curr_max: Optional[int | float] = None
        self.freq: dict[int | float] = {}
        self.max_freq: int = 0
        self.modes: Set[int | float] = set()

        self.k = k
        self.q: Deque[int | float] = deque()
        self.ktotal: float = 0
        self.maxdq: Deque[int | float] = deque()
        self.kfreq: dict[int | float] = {}
        self.bucket: dict[int, Set[int | float]] = defaultdict(set)
        self.kmax_freq: int = 0

    def insert(self, x: int | float) -> None:
        self.n += 1
        self.total += x

        if self.curr_max is None or x > self.curr_max:
            self.curr_max = x

        c = self.freq.get(x,0) + 1
        self.freq[x] = c 

        if c > self.max_freq:
            self.max_freq = c 
            self.modes = {x}
        elif c == self.max_freq:
            self.modes.add(x)

        # k window
        self.q.append(x)
        self.ktotal += x

        while self.maxdq and self.maxdq[-1] < x:
            self.maxdq.pop()
        self.maxdq.append(x)

        old = self.kfreq.get(x,0)
        new = old + 1
        self.kfreq[x] = new
        if old > 0:
            self.bucket[old].discard(x)
        self.bucket[new].add(x)
        if new > self.kmax_freq:
            self.kmax_freq = new

        if len(self.q) > self.k:
            y = self.q.popleft()
            self.ktotal -= y

            if self.maxdq and self.maxdq[0] == y:
                self.maxdq.popleft()

            oldf = self.kfreq[y]
            self.bucket[oldf].discard(y)
            if oldf == 1:
                del self.kfreq[y]
            else:
                self.kfreq[y] = oldf - 1
                self.bucket[oldf - 1].add(y)

            while self.kmax_freq > 0 and not self.bucket[self.kmax_freq]:
                self.kmax_freq -= 1

    def insert_many(self, xs: Iterable[int | float]) -> None:
        for x in xs:
            self.insert(x)

    def mean(self) -> Optional[float]:
        return None if self.n == 0 else self.total/self.n
    
    def max(self) -> Optional[int | float]:
        return self.curr_max
    
    def modes_all(self) -> List[int | float]:
        return list(self.modes)
    
    def kmean(self) -> Optional[float]:
        return None if not self.q else self.ktotal/len(self.q)

    def kmax(self) -> Optional[int | float]:
        return self.maxdq[0] if self.maxdq else None
    
    def k_modes(self) -> List[int | float]:
        if self.kmax_freq == 0:
            return []
        return list(self.bucket[self.kmax_freq])
    
s = DataStream(5)
s.insert_many([3,1,2,3,2,2,5,1,1,4,2,1])
print(s.mean())
print(s.max())
print(s.modes_all())
print(s.kmean())
print(s.kmax())
print(s.k_modes())

2.25
5
[1, 2]
1.8
4
[1]


In [42]:
def profitable_buys_no_K(buy, sell):
    n = len(buy)
    if n != len(sell):
        raise ValueError("buy 与 sell 长度必须相等")
    if n <= 1:
        return []

    # 后缀最大：suf_max[i] = max(sell[i..n-1])
    suf_max = [0]*n
    cur = float("-inf")
    for i in range(n-1, -1, -1):
        cur = max(cur, sell[i])
        suf_max[i] = cur

    profitable_indices = []
    # 对于每个 i，看未来的最大卖价是否大于 buy[i]
    for i in range(n-1):  # 最后一个 i 没有未来
        if suf_max[i+1] > buy[i]:
            profitable_indices.append(i)
    return profitable_indices

def profitable_with_earliest_sell(buy, sell):
    n = len(buy)
    if n != len(sell):
        raise ValueError("长度不等")
    if n <= 1:
        return []

    suf_max = [0]*n
    suf_pos = [0]*n  # 对应最大值的位置
    cur_val, cur_pos = float("-inf"), -1
    for i in range(n-1, -1, -1):
        if sell[i] >= cur_val:
            cur_val, cur_pos = sell[i], i
        suf_max[i] = cur_val
        suf_pos[i] = cur_pos

    res = []
    for i in range(n-1):
        if suf_max[i+1] > buy[i]:
            res.append((i, suf_pos[i+1]))  # (买入时刻i, 最早盈利卖出时刻j)
    return res

In [43]:
from collections import deque

def profitable_buys_with_K(buy, sell, K):
    n = len(buy)
    if n != len(sell):
        raise ValueError("buy 与 sell 长度必须相等")
    if n <= 1 or K <= 0:
        return []

    dq = deque()  # 存储 sell 的下标，保持 sell 值递减
    res = []

    # 让窗口是 [i+1, i+K]，我们在遍历 i 时，需要先把 i+K 加入候选，再把 <(i+1) 的下标移除
    # 为了方便，先预热把 [1..min(K, n-1)] 放入队列，供 i=0 使用
    def push(j):
        while dq and sell[dq[-1]] <= sell[j]:
            dq.pop()
        dq.append(j)

    right = 1
    while right < n and right <= K:
        push(right)
        right += 1

    for i in range(n-1):  # i = n-1 时无未来
        # 窗口左边界是 i+1，先弹出左侧过期元素
        while dq and dq[0] <= i:
            dq.popleft()

        # 窗口右侧，尝试扩展到 i+K（如果我们之前没扩到位）
        while right < n and right <= i + K:
            push(right)
            right += 1

        # 窗口最大值在 dq[0]
        if dq and sell[dq[0]] > buy[i]:
            res.append(i)

    return res

In [46]:
buy  = [5, 4, 3, 6, 2, 8]
sell = [3, 5, 4, 2, 9, 1]
K = 2

profitable_buys_with_K(buy, sell, 2)  # [2, 3]

[2, 3]

In [49]:
buy  = [5, 3, 4]
sell = [6, 7, 2]
K = 10

profitable_buys_with_K(buy, sell, K)  # [0, 1]
profitable_buys_no_K(buy, sell)       # [0, 1]

[0]

In [50]:
class OrderBook:
    def __init__(self, symbol):
        """
        初始化 OrderBook
        symbol: 股票代码
        """
        self.symbol = symbol
        self.bids = []  # [(price, size)]
        self.asks = []  # [(price, size)]

    def load_quotes(self, bids, asks):
        """
        读取经纪商报价
        bids, asks: list of (price, size)
        bids: 买价从高到低排序
        asks: 卖价从低到高排序
        """
        # 防御性排序
        self.bids = sorted(bids, key=lambda x: -x[0])
        self.asks = sorted(asks, key=lambda x: x[0])

    def buy_price(self, num_shares):
        """
        模拟买入 num_shares 股，从最低 ask 开始逐层成交。
        返回平均成交价（加权平均）
        """
        shares_left = num_shares
        total_cost = 0.0

        for price, size in self.asks:
            trade_size = min(size, shares_left)
            total_cost += trade_size * price
            shares_left -= trade_size
            if shares_left == 0:
                break

        if shares_left > 0:
            raise ValueError(f"市场流动性不足，只能买到 {num_shares - shares_left} 股。")

        return total_cost / num_shares

    def sell_price(self, num_shares):
        """
        模拟卖出 num_shares 股，从最高 bid 开始逐层成交。
        返回平均成交价（加权平均）
        """
        shares_left = num_shares
        total_revenue = 0.0

        for price, size in self.bids:
            trade_size = min(size, shares_left)
            total_revenue += trade_size * price
            shares_left -= trade_size
            if shares_left == 0:
                break

        if shares_left > 0:
            raise ValueError(f"市场流动性不足，只能卖出 {num_shares - shares_left} 股。")

        return total_revenue / num_shares


In [51]:
book = OrderBook("AAPL")

# 模拟 Broker 报价（price, shares）
bids = [(199.5, 300), (199.4, 400), (199.3, 500)]  # 买单
asks = [(199.6, 200), (199.7, 300), (199.8, 600)]  # 卖单

book.load_quotes(bids, asks)

print("买入500股平均价:", book.buy_price(500))
print("卖出600股平均价:", book.sell_price(600))

买入500股平均价: 199.66
卖出600股平均价: 199.45


In [ ]:
def flatten(lst):
    result = []
    for x in lst:
        if isinstance(x, list):
            result.extend(flatten(x))
        else:
            result.append(x)
    return result

nested = [1, [2, [3, 4], 5], [6, 7]]
print(flatten(nested))  # [1, 2, 3, 4, 5, 6, 7]
# O(n), not appropriate for deep nested lists

[1, 2, 3, 4, 5, 6, 7]


In [54]:
def flatten_iter(lst):
    stack = list(lst)[::-1]
    while stack:
        x = stack.pop()
        if isinstance(x, list):
            stack.extend(x[::-1])
        else:
            yield x

# 示例
nested = [1, [2, [3, [4, 5]]], [6, 7]]
print(list(flatten_iter(nested)))  # [1, 2, 3, 4, 5, 6, 7]
# Space Complexity O(depth)


[1, 2, 3, 4, 5, 6, 7]


In [55]:
lst = [1,1,2,2,3,4,5,5,6,7,8]

list(set(lst))

[1, 2, 3, 4, 5, 6, 7, 8]

In [56]:
list(dict.fromkeys(lst))

[1, 2, 3, 4, 5, 6, 7, 8]

In [57]:
def remove_dup(lst: list) -> list:
    seen = set()
    results = []
    for x in lst:
        if x not in seen:
            seen.add(x)
            results.append(x)
    return results

In [59]:
remove_dup(lst)

[1, 2, 3, 4, 5, 6, 7, 8]

In [10]:
from typing import Optional, Iterable, List, Deque
from collections import deque, defaultdict

class Datastream:
    def __init__(self, k) -> None:
        self.n: int = 0
        self.total: float = 0
        self.curr_max: Optional[int | float] = None
        self.freq: dict[int | float] = {}
        self.max_freq: int = 0
        self.modes: Set[int | float] = set()

        self.k: int = k
        self.q: Deque[int | float] = deque()
        self.ktotal: float = 0
        self.maxdq: Deque[int | float] = deque()
        self.kfreq: dict[int | float] = {}
        self.bucket: dict[int, Set[int | float]] = defaultdict(set)
        self.kmax_freq: int = 0

    def insert(self, x: int | float) -> None:
        self.n += 1
        self.total += x

        if self.curr_max is None or x > self.curr_max:
            self.curr_max = x 

        c = self.freq.get(x,0) + 1
        self.freq[x] = c 

        if c > self.max_freq:
            self.max_freq = c
            self.modes = {x}
        elif c == self.max_freq:
            self.modes.add(x)

        # k window
        self.q.append(x)
        self.ktotal += x

        while self.maxdq and self.maxdq[-1] < x:
            self.maxdq.pop()
        self.maxdq.append(x)

        old = self.kfreq.get(x, 0)
        new = old + 1
        self.kfreq[x] = new
        if old > 0:
            self.bucket[old].discard(x)
        self.bucket[new].add(x)
        if new > self.kmax_freq:
            self.kmax_freq = new
        
        if len(self.q) > self.k:
            y = self.q.popleft()
            self.ktotal -= y

            if self.maxdq and self.maxdq[0] == y:
                self.maxdq.popleft()

            oldf = self.kfreq[y]
            self.bucket[oldf].discard(y)
            if oldf == 1:
                del self.kfreq[y]
            else:
                self.kfreq[y] = oldf - 1
                self.bucket[oldf - 1].add(y)

            while self.kmax_freq > 0 and not self.bucket[self.kmax_freq]:
                self.kmax_freq -= 1

    def insert_many(self, xs: Iterable[int | float]) -> None:
        for x in xs:
            self.insert(x)

    def mean(self) -> Optional[int | float]:
        return None if self.n == 0 else self.total/self.n
    
    def max(self) -> Optional[int | float]:
        return self.curr_max
    
    def mode_all(self) -> List[int | float]:
        return list(self.modes)
    
    def kmean(self) -> Optional[int | float]:
        return None if not self.q else self.ktotal/len(self.q)
    
    def kmax(self) -> Optional[int | float]:
        return self.maxdq[0] if self.maxdq else None
    
    def k_modes(self) -> List[int | float]:
        if self.kmax_freq == 0: 
            return []
        return list(self.bucket[self.kmax_freq])
    
s = Datastream(5)
s.insert_many([3,1,2,3,2,2,5,3,2,1,3])
print(s.mean())
print(s.max())
print(s.mode_all())
print(s.kmean())
print(s.kmax())
print(s.k_modes())


2.4545454545454546
5
[2, 3]
2.8
5
[3]


In [ ]:
import sys
from collections import Counter
import math

freq = Counter()

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    try:
        val = float(line)
    except:
        continue

    if math.isnan(val) or val <= 0:
        continue

    intval = int(val)
    freq[intval] += 1

if not freq:
    print('0:')
else:
    max_count = max(freq.values())
    most_freq = sorted([k for k, v in freq.items() if v == max_count])
    print(f"{max_count}:{','.join(map(str, most_freq))}")

In [2]:
def maximumOccuringCharacter(text):
    freq = {}

    for ch in text:
        freq[ch] = freq.get(ch, 0) + 1

    max_count = max(freq.values())

    for ch in text: 
        if freq[ch] == max_count:
            return ch

In [4]:
maximumOccuringCharacter('abbbaacc')

'a'

In [5]:
import re

num_pattern = r'[+-]?(0|[1-9]\d*)(\.\d+)?'

coord_regex = re.compile(rf'^\(({num_pattern}), ({num_pattern})\)$')

def isValid(coordinates):
    for s in coordinates:
        m = coord_regex.match(s)
        if not m:
            print('Invalid')
            continue
        x_str, y_str = m.group(1), m.group(4)
        try:
            x = float(x_str)
            y = float(y_str)
        except:
            print('Invalid')
            continue
        if not (-90 <= x <= 90 and -180 <= y <= 180):
            print('Invalid')
            continue
        print('Valid')


In [6]:
isValid(['(90, 180)', '(+90, -180)', '(90, 180)', '(90,0, 180.1)', '(855, 95W)'])

Valid
Valid
Valid
Invalid
Invalid


In [7]:
def arraySum(numbers):
    return sum(numbers)

In [8]:
arraySum([3,13,4,11,9])

40

In [20]:
import ivolatility as ivol

ivol.setLoginParams(apiKey='neHi06s1WwGL40Uc')

In [21]:
# EOD Equity Options Raw IV (search by parameters)
getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')

marketData = getMarketData(symbol='SPX',tradeDate='2024-08-05',dteFrom=0,dteTo=0,deltaFrom=-1000,deltaTo=1000,cp='C')

In [22]:
marketData

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,...,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,is_settlement
0,2024-08-05,SPXW 240805C02200000,0,9327,2024-08-05,C,2200.0,2990.00,2990.00,2990.00,...,0.0,0.0,0.0,0.0,3004.20,2980.3,5192.005,-57.63,133236043,0
1,2024-08-05,SPXW 240805C02400000,0,9327,2024-08-05,C,2400.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2804.20,2780.3,5192.005,-53.78,133236045,0
2,2024-08-05,SPXW 240805C02600000,0,9327,2024-08-05,C,2600.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2604.20,2580.3,5192.005,-49.92,133236047,0
3,2024-08-05,SPXW 240805C02800000,0,9327,2024-08-05,C,2800.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2404.20,2380.3,5192.005,-46.07,133236049,0
4,2024-08-05,SPXW 240805C03000000,0,9327,2024-08-05,C,3000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2198.80,2180.3,5192.005,-42.22,133236051,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2024-08-05,SPXW 240805C06400000,0,9327,2024-08-05,C,6400.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,23.27,133236161,0
209,2024-08-05,SPXW 240805C06600000,0,9327,2024-08-05,C,6600.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,27.12,133236163,0
210,2024-08-05,SPXW 240805C06800000,0,9327,2024-08-05,C,6800.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,30.97,133236165,0
211,2024-08-05,SPXW 240805C07000000,0,9327,2024-08-05,C,7000.0,0.05,0.05,0.05,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,34.82,133236167,0


In [23]:
import requests

In [40]:
api_key = "neHi06s1WwGL40Uc"

url = (
    "https://restapi.ivolatility.com/equities/eod/stock-opts-by-param"
)

params = {
    "apiKey": api_key,
    "symbol": "AAPL",
    "tradeDate": "2025-12-02",
    "dteFrom": 3,
    "dteTo": 3,
    "deltaFrom": -100,
    "deltaTo": 100,
    "cp": "C"
}

r = requests.get(url, params=params)
data = r.json()

In [41]:
if data['data']:
    print('True')
else:
    print('False')

True


In [49]:
type(data)

dict

In [56]:
import json
print(json.dumps(data, indent=2))

{
  "status": {
    "executionTime": 64,
    "recordsFound": 75,
    "code": "COMPLETE",
    "urlForDetails": null
  },
  "query": {
    "requestUUID": "a97288b3-97b8-4152-87be-6d748c67549f"
  },
  "data": [
    {
      "c_date": "2025-12-02",
      "option_symbol": "AAPL  251205C00110000",
      "dte": 3,
      "stocks_id": 799,
      "expiration_date": "2025-12-05",
      "call_put": "C",
      "price_strike": 110.0,
      "price_open": 174.96,
      "price_high": 175.6,
      "price_low": 174.81,
      "price": 176.1,
      "volume": 10,
      "openinterest": 7,
      "iv": 0.766583,
      "delta": 1.0,
      "preiv": -1.0,
      "gamma": 0.0,
      "theta": -0.012469,
      "vega": 0.0,
      "rho": 0.009038,
      "Ask": 177.9,
      "Bid": 174.3,
      "underlying_price": 286.19,
      "calc_OTM": -61.56,
      "option_id": 141941085,
      "is_settlement": 0
    },
    {
      "c_date": "2025-12-02",
      "option_symbol": "AAPL  251205C00120000",
      "dte": 3,
      "stocks_i

In [57]:
from IPython.display import JSON
JSON(data)

<IPython.core.display.JSON object>